<a href="https://colab.research.google.com/github/Mikhthad/Malaria-/blob/main/malaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install tensorflow-datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow_datasets as tfds
import pandas as pd

In [ ]:
pip list

In [ ]:
import os

os.environ['TFDS_DATA_DIR'] = "C:\\tensorflow_datasets"

In [ ]:
dataset,dataset_info  = tfds.load('malaria',data_dir="C:\\tensorflow_datasets",with_info=True,as_supervised=True,shuffle_files=True,split=['train'])

In [ ]:
dataset

In [ ]:
dataset_info

In [ ]:
len(dataset)

In [ ]:
def splits(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO):
    DATASET_SIZE = len(dataset)
    train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))
    val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
    val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))
    test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
    return train_dataset,val_dataset,test_dataset

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset,val_dataset,test_dataset = splits(dataset[0],TRAIN_RATIO,VAL_RATIO,TEST_RATIO)

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i, (image,label) in enumerate(train_dataset.take(16)):
    ax = plt.subplot(4,4,i+1)
    plt.imshow(image)
    plt.title(dataset_info.features['label'].int2str(label))
    plt.axis("off")

In [ ]:
im_size = 224

In [ ]:
image

In [ ]:
plt.imshow(image)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
IM_SIZE = 224

In [ ]:
def resize_rescale(image,label):
    return tf.image.resize(image,(IM_SIZE,IM_SIZE))/255.0,label

In [ ]:
image

In [ ]:
train_dataset

In [ ]:
BATCH_SIZE = 32
train_dataset  =train_dataset.map(resize_rescale).shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset  =val_dataset.map(resize_rescale).shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset  =test_dataset.map(resize_rescale).shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_dataset

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import InputLayer
from tensorflow.keras import layers

In [ ]:
model = tf.keras.Sequential([
    InputLayer(input_shape=(IM_SIZE, IM_SIZE, 3)),
    layers.Conv2D(filters=6, kernel_size=5, strides=1, padding="valid"),
    BatchNormalization(),
    layers.ReLU(),

    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Conv2D(filters=16, kernel_size=5, strides=1, padding="valid"),
    BatchNormalization(),
    layers.ReLU(),

    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Flatten(),
    layers.Dense(1000),
    BatchNormalization(),
    layers.ReLU(),

    layers.Dense(100),
    BatchNormalization(),
    layers.ReLU(),

    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=BinaryCrossentropy(),
    metrics=['accuracy']
)


In [ ]:
history = model.fit(train_dataset,epochs=10,validation_data=val_dataset,verbose=1)

In [ ]:
y_pred = model.predict(test_dataset)
y_pred

In [ ]:
import numpy as np
pred = np.where(y_pred>0.5,1,0)
pred

In [ ]:
a = [100,3100,400,200,500,600]

In [ ]:
np.argmax(a)

In [ ]:
# accuracy_score(test_dataset,y_pred)

In [ ]:
test_dataset

In [ ]:
history.history

In [ ]:
model.evaluate(test_dataset)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
true_labels = np.concatenate([y for x, y in test_dataset], axis=0)
print(classification_report(true_labels,pred))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(true_labels,pred)

In [ ]:
model.count_params()

In [ ]:
model.save('Malaria_detection.h5')